In [1]:
import cv2
import mediapipe as mp

In [2]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [3]:
face = mp_face_detection.FaceDetection(min_detection_confidence=0.75)

In [4]:
cap = cv2.VideoCapture(0)

In [5]:
def fancyDraw(image ,bbox ,l=30 ,t=5 ,rt=1):
    x,y,w,h = bbox
    
    # diagonal point(bottom right corner point)
    x1,y1 = x+w ,y+h
    
    cv2.rectangle(image ,bbox ,(0,255,0) ,rt)
    
    # top left
    cv2.line(image ,(x,y) ,(x+l,y) ,(0,0,255) ,t)
    cv2.line(image ,(x,y) ,(x,y+l) ,(0,0,255) ,t)
    
    # top right
    cv2.line(image ,(x1,y) ,(x1-l,y) ,(0,0,255) ,t)
    cv2.line(image ,(x1,y) ,(x1,y+l) ,(0,0,255) ,t)
    
    # bottom left
    cv2.line(image ,(x,y1) ,(x+l,y1) ,(0,0,255) ,t)
    cv2.line(image ,(x,y1) ,(x,y1-l) ,(0,0,255) ,t)
    
    # bottom right
    cv2.line(image ,(x1,y1) ,(x1-l,y1) ,(0,0,255) ,t)
    cv2.line(image ,(x1,y1) ,(x1,y1-l) ,(0,0,255) ,t)
    
    return image

In [6]:
def findFaces(results ,image ,draw=True):
    h,w = image.shape[:2]
    bboxes = []
    
    if results.detections:
        for id,detection in enumerate(results.detections):
            bboxC = detection.location_data.relative_bounding_box
            bbox = int(bboxC.xmin*w) , int(bboxC.ymin*h) , int(bboxC.width*w) , int(bboxC.height*h)
            
            bboxes.append([id ,bbox ,detection.score])
            
            if draw :
                image = fancyDraw(image ,bbox)
                #cv2.rectangle(image ,bbox ,(0,255,0) ,4 )
                cv2.putText(image ,f'Confi : {int(detection.score[0]*100)}%' ,(bbox[0],bbox[1]-20) ,cv2.FONT_HERSHEY_PLAIN ,2 ,(0,255,0) ,2)

    
    return image,bboxes

In [7]:
while True:
    success, image = cap.read()
    h,w = image.shape[:2]
    
    if success:
        
        imageRGB = cv2.cvtColor(image ,cv2.COLOR_BGR2RGB)
        results = face.process(imageRGB)
        
        image,bboxes = findFaces(results ,image)
        
        cv2.imshow('MediaPipe Face Detection', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
              break

cap.release()
cv2.destroyAllWindows()